In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists,druglists
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Process raw CSV files

In [5]:
# create_pegmed()

In [6]:
# create_pegprod()

In [7]:
# create_prescriptions()

In [8]:
# create_consultations()

In [9]:
# create_clinicals()

In [10]:
# create_tests()

In [11]:
# create_referrals()

In [12]:
# create_immunisations()

In [13]:
# create_medcoded_entries()

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, female gender, insomnia) used in the logistic regression*

In [14]:
# specify subtype of dementia we're interested in - 'vascular','alzheimers' or 'all_dementia'
subtype = 'all_dementia' 

In [15]:
all_encounters = get_all_encounters()

In [18]:
all_entries = get_all_entries(all_encounters)

In [19]:
pt_features = create_pt_features()

In [20]:
pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [ ]:
# pt_features = only_include_specific_dementia_subtype(pt_features,subtype=subtype) #does nothing if we're interested in all dementia

In [ ]:
# pt_features = add_data_start_and_end_dates(all_encounters,pt_features)

In [53]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'earliest_sysdate'],
      dtype='object')

In [54]:
len(pt_features)

94514

In [55]:
latest_sysdates = all_encounters.groupby('patid')['sysdate'].max().reset_index()
latest_sysdates.rename(columns={'sysdate':'data_end'},inplace=True)
pt_features = pd.merge(pt_features,latest_sysdates,how='left')

In [56]:
earliest_sysdates = all_encounters.groupby('patid')['sysdate'].min().reset_index()
earliest_sysdates.rename(columns={'sysdate':'earliest_sysdate'},inplace=True)
pt_features = pd.merge(pt_features,earliest_sysdates,how='left')

In [57]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'earliest_sysdate'],
      dtype='object')

In [58]:
converted_code_entries = all_encounters[all_encounters['medcode']==14] 
#finds all years where medcode=14 (i.e. a converted code - where the GP practice appears to have overwritten all specific consultation codes with an uninformative 'Converted code' medcode)

In [59]:
converted_code_entries.sample(5)

,eventdate,medcode,patid,prodcode,sysdate,type
48686436,2003-08-19,14.0,3456106,NaN,2006-04-21,2
71553411,2010-02-10,14.0,10177656,NaN,2011-09-12,2
67571847,1999-06-28,14.0,10725515,NaN,2007-01-15,2
71239591,NaT,14.0,5748647,NaN,2001-08-21,2
61141849,1995-09-19,14.0,5683380,NaN,2000-10-20,2


In [60]:
latest_converted_code_entries = converted_code_entries.groupby('patid')['sysdate'].max().reset_index()

In [63]:
latest_converted_code_entries.columns = ['patid','sysdate_of_final_converted_code']

In [64]:
latest_converted_code_entries.sample(15)

,patid,sysdate_of_final_converted_code
15073,28082632,2007-03-27
15080,28164200,2000-12-22
14312,20079481,2008-09-26
8915,7234589,2008-05-01
10225,8952246,2000-08-01
9233,7605009,2000-09-05
13973,18249647,2010-11-13
4862,3413495,2006-12-11
12287,12762564,2005-02-16
687,473524,2007-04-20


In [66]:
pt_features = pd.merge(pt_features,latest_converted_code_entries,how='left')

In [68]:
pt_features.sample(15)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,earliest_sysdate,sysdate_of_final_converted_code
74999,7273496,24,496,0,2010-08-18,True,1917.0,2013-05-04,2007-09-28,2007-09-28
41109,5592266,35,266,0,2011-05-05,True,1917.0,2011-09-16,2000-05-31,NaT
23416,5893158,15,158,1,2006-05-30,True,4357.0,2011-10-20,1997-10-22,NaT
41035,4474265,19,265,1,NaT,False,NaN,2006-11-20,2002-10-15,NaT
14185,16810100,20,100,1,NaT,False,NaN,2005-10-31,2002-08-07,NaT
11937,10227088,23,088,0,2010-02-25,True,1916.0,2011-03-16,1996-06-08,NaT
88490,20221633,21,633,1,NaT,False,NaN,2010-05-17,2007-10-31,2007-10-31
26873,11672178,22,178,1,2009-06-30,True,7664.0,2013-04-18,1998-12-14,NaT
16955,15602120,45,120,0,NaT,False,NaN,2010-11-26,1995-12-12,NaT
40253,3462260,37,260,1,NaT,False,NaN,2013-05-01,2001-08-21,NaT


In [ ]:
resampled_entries = all_encounters.set_index('eventdate').groupby('patid').resample('AS').size()

In [ ]:
resampled_entries.head(15)

In [ ]:
resampled_entries2 = resampled_entries.reset_index()

In [ ]:
resampled_entries2.columns = ['patid','year','consultation_count']

In [ ]:
resampled_entries2.head(15)

In [ ]:
resampled_entries3 = resampled_entries2.loc[resampled_entries2['consultation_count']>=10]

In [ ]:
resampled_entries3.head(15)

In [ ]:
resampled_entries4 = resampled_entries3.groupby('patid').year.min().reset_index()

In [ ]:
resampled_entries4.head(15)

In [ ]:
resampled_entries4['year']=resampled_entries4['year']+pd.Timedelta(days=365) 

In [ ]:
resampled_entries4.columns=['patid','start_of_year_after_earliest_year_with_>10_consultations']

In [ ]:
resampled_entries4.head(15)

In [ ]:
pt_features = pd.merge(pt_features,resampled_entries4,how='left')

In [ ]:
pt_features.sort_values(by='start_of_year_after_earliest_year_with_>10_consultations')

In [ ]:
dont_use_earliest_sysdate_mask = (pt_features['start_of_year_after_earliest_year_with_>10_consultations']>pt_features['earliest_sysdate']) &
        (pt_features['start_of_year_after_earliest_year_with_>10_consultations'] > pt_features['sysdate_of_final_converted_code'])) 

In [ ]:
pt_features['data_end']=pt_features['earliest_sysdate']

In [ ]:
pt_features.loc[dont_use_earliest_sysdate_mask,'data_end']=pt_features['start_of_year_after_earliest_year_with_>10_consultations'

In [ ]:
# pts_without_any_events = pt_features.loc[pd.isnull(pt_features['data_start'])]
# print('There are {0} patients without any events'.format(len(pts_without_any_events)))
# if len(pts_without_any_events)>0:
#     pts_without_any_events.loc[:,'reason_for_removal']='Pt did not have any events'
#     pts_without_any_events.to_csv('data/pt_data/removed_patients/pts_without_any_events.csv',index=False)

# pt_features = pt_features.loc[pd.notnull(pt_features['data_start'])]

In [ ]:
# pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',index=False)

In [ ]:
# for window in sd.exposure_windows:
#     print(window['name'],' being matched')    
#     pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
#     pt_features = match_cases_and_controls(pt_features,window)
#     pt_features = delete_unmatched_cases_and_controls(pt_features)
#     pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

In [ ]:
# len(pt_features)

## Add derived variables to pt_features 
*e.g. insomnia count, presence of diabetes, consultation count*

In [ ]:
# medcoded_entries = pd.read_hdf('hdf/medcoded_entries.hdf')

In [ ]:
# prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [ ]:
# pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+sd.exposure_windows[1]['name']+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
# create_pdd_for_each_drug(prescriptions,druglists.all_druglists,pt_features,sd.exposure_windows[1])

In [ ]:
# # Add condition status (e.g. insomnia count, presence of diabetes, presence of stroke)
# for window in sd.exposure_windows:
#     print(window['name'],'...')
#     pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
#     pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,window,codelists.all_codelists)
#     pt_features = create_PDD_columns_for_each_pt(pt_features,window,druglists.all_druglists,prescriptions)
#     pt_features = get_consultation_count(pt_features,all_encounters,window)
#     pt_features = create_quantiles_and_booleans(pt_features)
#     pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

In [ ]:
# old_pts = pd.read_csv('data/pt_data/removed_patients/removed_unmatched_patients.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [ ]:
# old_pts.sample(15)

# Sandbox

In [47]:
pd.options.display.max_rows = 10000

In [48]:
patid = 3458

In [45]:
hist = get_patient_history(all_entries,patid).sort_values(by='eventdate')

In [49]:
hist.sort_values(by='sysdate')

,patid,medcode,prodcode,eventdate,sysdate,type,description
1,3458,NaN,NaN,1942-01-01,2005-01-24,consultation,NaN
984,3458,19262.0,NaN,2003-11-13,2005-01-24,clinical,Chest pain not present
983,3458,6326.0,NaN,2003-11-13,2005-01-24,clinical,Breathless - moderate exertion
61,3458,NaN,NaN,2003-11-11,2005-01-24,consultation,NaN
43,3458,NaN,NaN,2003-10-31,2005-01-24,consultation,NaN
982,3458,2269.0,NaN,2003-10-31,2005-01-24,clinical,Cardiological referral
42,3458,NaN,NaN,2003-10-20,2005-01-24,consultation,NaN
985,3458,31134.0,NaN,2003-11-13,2005-01-24,clinical,Medication counselling
1885,3458,6.0,NaN,2003-10-20,2005-01-24,immunisation,Influenza vaccination
981,3458,91.0,NaN,2003-09-18,2005-01-24,clinical,Referral for further care
